# **SetUp**

In [3]:
!nvidia-smi

Tue Feb 17 05:07:12 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 NVL                Off |   00000000:3D:00.0 Off |                    0 |
| N/A   42C    P0             74W /  400W |       1MiB /  95830MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# **ZO**

- L0: Baseline `None`

- L1: 增量 checkpoint `INCR_CKPT=1`

- L2: + CPU Shadow `INCR_CKPT=1 ENABLE_SHADOW=1`

- L3: + 即时恢复 `INCR_CKPT=1 ENABLE_SHADOW=1 INSTANT_RECOVER=1 GPU_FAIL_STEP=200`

- GPU 故障注入 (仅观察，不恢复) `GPU_FAIL_STEP=200`

In [ ]:
# 简单staging
!MODEL=Qwen/Qwen3-1.7B \
    TRAIN_NAME=Test_staging_5 TASK=SST2 MODE=ft LR=1e-7 EPS=1e-3 \
    STEPS=1000 EVAL_STEPS=100 SAVE_STEPS=100 DO_EVAL=0 \
    INCR_CKPT=1 GPU_FAIL_STEP=200 \
    bash /home/users/u0001609/NonStopZO2/example/mezo_runner/mezo.sh

========== Configuration ==========
TAG: mezo-ft-1e-6-1e-4-0
BS: 16, LR: 1e-6, EPS: 1e-4, SEED: 0
STEPS: 1000, EVAL_STEPS: 100, SAVE_STEPS: 100
MODE: ft, DO_EVAL: 0
--- Experiment Level ---
L1 INCR_CKPT: 1
L2 ENABLE_SHADOW: 0
L3 INSTANT_RECOVER: 0
GPU_FAIL_STEP: 200
Extra args:  --incremental_checkpoint --gpu_fail_step 200 --no_eval 


2026-02-19 05:14:12,644 - INFO - PyTorch version 2.10.0 available.
OurArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
enable_shadow=False,
eos_token=<EOS_TOKEN>,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True

In [ ]:
# 即时恢复
!MODEL=Qwen/Qwen3-1.7B \
    TASK=SST2 MODE=ft LR=1e-3 EPS=1e-3 \
    STEPS=1000 EVAL_STEPS=100 SAVE_STEPS=100 DO_EVAL=0 \
    INCR_CKPT=1 ENABLE_SHADOW=1 FAIL_STEP=200 \
    bash /home/users/u0001609/NonStopZO2/example/mezo_runner/mezo.sh

# **Resume**

In [ ]:
# 简单staging
!MODEL=Qwen/Qwen3-1.7B \
    TRAIN_NAME=Test_staging_2 TASK=SST2 MODE=ft LR=1e-3 EPS=1e-3 \
    STEPS=1000 EVAL_STEPS=100 SAVE_STEPS=100 DO_EVAL=0 \
    RESUME_CKPT="/lvs0/rccs-hpbdrt/minqiu/ZO_ckpt/Test_staging_1-SST2-Qwen3-1.7B-mezo-ft-1e-3-1e-3-0/checkpoint-700" \
    bash /home/users/u0001609/NonStopZO2/example/mezo_runner/mezo.sh

# **Read CKPT**

In [ ]:
import torch
import json

# 读取 .pt 文件
pt_path = "/lvs0/rccs-hpbdrt/minqiu/ZO_ckpt/Test_staging_4-SST2-Qwen3-1.7B-mezo-ft-1e-6-1e-4-0/checkpoint-200/zo_replay_meta.pt"
meta = torch.load(pt_path, map_location='cpu', weights_only=True)
print("=== zo_replay_meta.pt ===")
print(meta)

# 读取 .json 文件
json_path = "/lvs0/rccs-hpbdrt/minqiu/ZO_ckpt/Test_staging_4-SST2-Qwen3-1.7B-mezo-ft-1e-6-1e-4-0/checkpoint-200/zo_replay_history.json"
with open(json_path, 'r') as f:
    history = json.load(f)
print("\n=== zo_replay_history.json ===")
print(history)

=== zo_replay_meta.pt ===
{'base_checkpoint': '/lvs0/rccs-hpbdrt/minqiu/ZO_ckpt/Test_staging_4-SST2-Qwen3-1.7B-mezo-ft-1e-6-1e-4-0/checkpoint-100', 'step': 200, 'is_zo_replay': True, 'num_updates': 100}

=== zo_replay_history.json ===
{'base_checkpoint': '/lvs0/rccs-hpbdrt/minqiu/ZO_ckpt/Test_staging_4-SST2-Qwen3-1.7B-mezo-ft-1e-6-1e-4-0/checkpoint-100', 'base_step': 100, 'current_step': 200, 'updates': [{'step': 100, 'seed': 813891745, 'grad': -107.4375, 'lr': 1e-06, 'wd': 0.0}, {'step': 101, 'seed': 623415497, 'grad': 153.75, 'lr': 1e-06, 'wd': 0.0}, {'step': 102, 'seed': 622028733, 'grad': -8.546875, 'lr': 1e-06, 'wd': 0.0}, {'step': 103, 'seed': 758744035, 'grad': -4.8828125, 'lr': 1e-06, 'wd': 0.0}, {'step': 104, 'seed': 420477197, 'grad': 2.44140625, 'lr': 1e-06, 'wd': 0.0}, {'step': 105, 'seed': 976125790, 'grad': -153.75, 'lr': 1e-06, 'wd': 0.0}, {'step': 106, 'seed': 738194223, 'grad': 87.875, 'lr': 1e-06, 'wd': 0.0}, {'step': 107, 'seed': 328740110, 'grad': -26.859375, 'lr': 